# Lab 4. Дослідження особливостей реалізації криптографічних систем

## Мета
Дослідити основні проблеми, що виникають при програмній реалізації
криптосистем, та запропонувати методи їх вирішення для User Endpoint terminal.

У роботі розглядаються:
- захист ключової інформації в оперативній пам’яті;
- контроль правильності функціонування криптографічних програм;
- порівняння інтерфейсів Crypto API та PKCS#11.


## Основні задачі реалізації криптосистем

При програмній реалізації криптосистем виникають такі ключові задачі:

1. **Захист ключової інформації в оперативній пам’яті**
   - ключі можуть бути прочитані через дамп пам’яті;
   - ключі можуть залишатись у пам’яті після завершення операцій.

2. **Контроль доступу до криптографічних операцій**
   - обмеження використання ключів лише дозволеними операціями;
   - розмежування прав доступу.

3. **Контроль правильності функціонування програми**
   - перевірка коректності обчислень;
   - виявлення збоїв або модифікації виконуваного коду.

4. **Залежність від механізмів захисту ОС**
   - програмні реалізації залежать від ізоляції процесів та прав доступу ОС.


In [ ]:
# DEMO: key as a regular object in RAM

import secrets

def generate_secret_key():
    return secrets.token_bytes(32)  # 256-bit symmetric key

secret_key = generate_secret_key()

print("Secret key length:", len(secret_key))
print("Secret key (hex):", secret_key.hex())


Secret key length: 32
Secret key (hex): f079d035bb43bfe2db7dd78d692d30129a86a783be853acb1c3b7db496150a05


## Мінімізація часу життя ключа в пам’яті

Одним з базових методів захисту ключів у пам’яті є мінімізація часу,
протягом якого ключ зберігається в оперативній пам’яті у відкритому вигляді.


In [ ]:
# DEMO: key zeroization (best-effort in Python)

def use_and_destroy_key():
    key = secrets.token_bytes(32)
    # emulate cryptographic operation
    _ = sum(key)
    # overwrite reference
    key = None
    return "Key used and reference destroyed"

use_and_destroy_key()


'Key used and reference destroyed'

У мовах з керуванням пам’яттю (зокрема Python) повна гарантія знищення
ключа неможлива, однак зменшення часу життя ключових даних суттєво
знижує ризики їх компрометації.


## Контроль правильності функціонування криптографічної програми

Для забезпечення коректної роботи криптографічних програм використовуються
спеціальні механізми контролю, які дозволяють виявляти:
- помилки реалізації;
- збої апаратури або середовища виконання;
- несанкціоновану модифікацію алгоритмів або даних.

У даній роботі розглянуто такі методи контролю:
1. самотестування (self-test) при запуску;
2. контроль цілісності результатів криптографічних операцій;
3. обробка помилкових або некоректних вхідних даних.


In [ ]:
# Self-test using known-answer test (KAT)

import hashlib
import hmac

def self_test():
    # Known values
    key = b"test_key"
    msg = b"test_message"
    expected = hmac.new(key, msg, hashlib.sha256).hexdigest()

    result = hmac.new(key, msg, hashlib.sha256).hexdigest()
    return result == expected

self_test()


True

Self-test виконується при запуску програми і гарантує, що базові
криптографічні операції працюють коректно до обробки реальних даних.


In [ ]:
# Integrity check with HMAC

def compute_hmac(key: bytes, data: bytes) -> bytes:
    return hmac.new(key, data, hashlib.sha256).digest()

def verify_hmac(key: bytes, data: bytes, tag: bytes) -> bool:
    return hmac.compare_digest(
        compute_hmac(key, data),
        tag
    )

key = secrets.token_bytes(32)
message = b"important data"

tag = compute_hmac(key, message)

verify_hmac(key, message, tag)


True

Використання HMAC дозволяє контролювати цілісність результатів
криптографічної обробки та виявляти будь-яку зміну даних.


In [ ]:
# Negative test: modified message

tampered_message = b"important DATA"

verify_hmac(key, tampered_message, tag)


False

У випадку модифікації даних контрольна перевірка не проходить,
що свідчить про коректну роботу механізму контролю цілісності.


## Порівняння інтерфейсів Crypto API та PKCS#11

При програмній реалізації криптографічних систем широко використовуються
стандартизовані криптографічні інтерфейси. Найпоширенішими з них є
Crypto API та PKCS#11.

Нижче наведено порівняння цих інтерфейсів з точки зору задач,
досліджених у даній лабораторній роботі.


### Загальна характеристика інтерфейсів

| Критерій | Crypto API | PKCS#11 |
|--------|------------|---------|
| Тип інтерфейсу | Платформо-залежний | Платформо-незалежний |
| Основне застосування | Програмні криптопровайдери ОС | Апаратні та програмні токени |
| Підтримка апаратних ключів | Обмежена | Повна |
| Контроль доступу до ключів | На рівні ОС | На рівні токена |
| Зберігання ключів | Пам’ять/сховище ОС | Усередині криптомодуля |


### Захист ключової інформації

**Crypto API:**
- ключі часто зберігаються у пам’яті процесу або в захищеному сховищі ОС;
- рівень захисту залежить від механізмів операційної системи;
- можливі атаки при компрометації ОС або процесу.

**PKCS#11:**
- ключі ніколи не залишають межі криптографічного модуля;
- операції виконуються всередині токена або HSM;
- значно знижений ризик компрометації ключів через дамп пам’яті.


### Контроль правильності функціонування

**Crypto API:**
- контроль реалізується на рівні програмного забезпечення;
- можливе використання self-test, контрольних векторів, HMAC;
- уразливий до підміни коду при компрометації ОС.

**PKCS#11:**
- криптомодулі зазвичай мають вбудовані self-test;
- контроль цілісності реалізований на рівні апаратури;
- сертифіковані HSM забезпечують відповідність стандартам безпеки.


### Придатність для User Endpoint terminal

Для середовища User Endpoint terminal характерні такі особливості:
- обмежений доступ до апаратних криптомодулів;
- використання стандартних програмних засобів ОС;
- залежність від програмного захисту.

У таких умовах Crypto API є більш простим та доступним рішенням,
тоді як PKCS#11 доцільний у системах з підвищеними вимогами
до захисту ключової інформації.


## Висновки до лабораторної роботи №4

1. Розглянуто основні проблеми програмної реалізації криптосистем,
   пов’язані із захистом ключів у оперативній пам’яті та контролем
   коректності функціонування криптографічних програм.
2. Показано, що у програмних реалізаціях ключові дані можуть бути
   скомпрометовані через пам’ять процесу, тому важливими є методи
   мінімізації часу життя ключів.
3. Реалізовано базові механізми контролю правильності роботи:
   self-test, контроль цілісності результатів за допомогою HMAC
   та обробку помилкових вхідних даних.
4. Проведено порівняння інтерфейсів Crypto API та PKCS#11 з точки зору
   захисту ключової інформації та контролю функціонування.
5. Встановлено, що Crypto API є зручним рішенням для User Endpoint terminal,
   тоді як PKCS#11 забезпечує вищий рівень безпеки за рахунок апаратної
   ізоляції ключів.
